## 1. Setup & Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Libraries imported successfully")

## 2. Load Datasets

In [ ]:
# Load data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
submission = pd.read_csv('../outputs/submission.csv')

print(f"📊 Training set: {train.shape}")
print(f"📊 Test set: {test.shape}")
print(f"📊 Submission: {submission.shape}")
print("\n" + "="*50 + "\n")

# Display first few rows
print("Training Data Sample:")
train.head()

## 3. Dataset Overview

In [ ]:
# Dataset information
print("📋 Dataset Information:")
print(f"   - Total samples: {len(train):,}")
print(f"   - Total features: {train.shape[1]}")
print(f"   - Component fractions: 5")
print(f"   - Component properties: 50 (5 components × 10 properties)")
print(f"   - Target blend properties: 10")
print(f"   - Missing values: {train.isnull().sum().sum()}")

train.info()

## 4. Component Fraction Analysis

In [ ]:
# Analyze component fractions
fraction_cols = [f'Component{i}_fraction' for i in range(1, 6)]
fractions = train[fraction_cols]

# Statistics
print("📊 Component Fraction Statistics:")
print(fractions.describe())

# Verify fractions sum to 1.0
fraction_sums = fractions.sum(axis=1)
print(f"\n✅ Fraction sums valid: {np.allclose(fraction_sums, 1.0)}")
print(f"   Mean sum: {fraction_sums.mean():.6f}")
print(f"   Std sum: {fraction_sums.std():.8f}")

In [ ]:
# Visualize component fraction distributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Component Fraction Distributions', fontsize=16, fontweight='bold')

for idx, col in enumerate(fraction_cols):
    ax = axes[idx // 3, idx % 3]
    ax.hist(train[col], bins=50, color=f'C{idx}', alpha=0.7, edgecolor='black')
    ax.set_title(col, fontweight='bold')
    ax.set_xlabel('Fraction')
    ax.set_ylabel('Frequency')
    ax.axvline(train[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {train[col].mean():.3f}')
    ax.legend()

# Remove empty subplot
fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

## 5. Blend Property Target Analysis

In [ ]:
# Target properties
target_cols = [f'BlendProperty{i}' for i in range(1, 11)]
targets = train[target_cols]

# Statistics
print("🎯 Blend Property Statistics:")
print(targets.describe())

# Value ranges
print("\n📊 Value Ranges:")
for col in target_cols:
    print(f"   {col}: [{train[col].min():.4f}, {train[col].max():.4f}] (range: {train[col].max() - train[col].min():.4f})")

In [ ]:
# Visualize target distributions
fig, axes = plt.subplots(2, 5, figsize=(18, 8))
fig.suptitle('Blend Property Distributions (Training Data)', fontsize=16, fontweight='bold')

for idx, col in enumerate(target_cols):
    ax = axes[idx // 5, idx % 5]
    ax.hist(train[col], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    ax.set_title(col, fontweight='bold')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    
    # Add statistics
    mean_val = train[col].mean()
    median_val = train[col].median()
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=1.5, label=f'Mean: {mean_val:.2f}')
    ax.axvline(median_val, color='green', linestyle='-.', linewidth=1.5, label=f'Median: {median_val:.2f}')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

## 6. Correlation Analysis

In [ ]:
# Correlation between blend properties
correlation_matrix = targets.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Blend Properties', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Identify highly correlated pairs
print("\n🔗 Highly Correlated Property Pairs (|r| > 0.7):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            print(f"   {correlation_matrix.columns[i]} ↔ {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

## 7. Component vs. Blend Property Relationships

In [ ]:
# Example: Component fractions vs. BlendProperty1
fig, axes = plt.subplots(1, 5, figsize=(18, 4))
fig.suptitle('Component Fractions vs. BlendProperty1', fontsize=14, fontweight='bold')

for idx, frac_col in enumerate(fraction_cols):
    axes[idx].scatter(train[frac_col], train['BlendProperty1'], alpha=0.3, s=10)
    axes[idx].set_xlabel(frac_col)
    axes[idx].set_ylabel('BlendProperty1')
    
    # Calculate correlation
    corr = train[frac_col].corr(train['BlendProperty1'])
    axes[idx].set_title(f'r = {corr:.3f}', fontsize=10)

plt.tight_layout()
plt.show()

## 8. Submission Predictions Analysis

In [ ]:
# Analyze submission predictions
pred_cols = [f'BlendProperty{i}' for i in range(1, 11)]
predictions = submission[pred_cols]

print("🔮 Submission Predictions Statistics:")
print(predictions.describe())

# Compare prediction ranges with training ranges
print("\n📊 Prediction Ranges vs. Training Ranges:")
for col in pred_cols:
    train_min, train_max = train[col].min(), train[col].max()
    pred_min, pred_max = predictions[col].min(), predictions[col].max()
    in_range = (pred_min >= train_min) and (pred_max <= train_max)
    status = "✅" if in_range else "⚠️"
    print(f"   {status} {col}:")
    print(f"      Training: [{train_min:.4f}, {train_max:.4f}]")
    print(f"      Predicted: [{pred_min:.4f}, {pred_max:.4f}]")

In [ ]:
# Visualize prediction distributions
fig, axes = plt.subplots(2, 5, figsize=(18, 8))
fig.suptitle('Prediction Distributions vs. Training Distributions', fontsize=16, fontweight='bold')

for idx, col in enumerate(pred_cols):
    ax = axes[idx // 5, idx % 5]
    
    # Training data
    ax.hist(train[col], bins=50, alpha=0.5, color='blue', label='Training', edgecolor='black')
    
    # Predictions
    ax.hist(predictions[col], bins=50, alpha=0.5, color='orange', label='Predictions', edgecolor='black')
    
    ax.set_title(col, fontweight='bold')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

## 9. Model Performance Estimation (If Validation Available)

In [ ]:
# Note: Since we don't have true test labels, this is a placeholder
# In practice, you would use cross-validation results from training

print("📈 Model Performance Insights:")
print("   - Public Leaderboard Score: 83.16")
print("   - Reference Cost (Public): 2.72")
print("   - Implied MAPE: ~1.85%")
print("\n   Formula: Score = 100 - 25 × (MAPE / Reference_Cost)")
print("   Solving: 83.16 = 100 - 25 × (MAPE / 2.72)")
print("   MAPE ≈ 1.83%")

# Calculate what MAPE would give specific scores
reference_cost = 2.72
scores = [100, 90, 85, 83.16, 80, 75, 70]

print("\n📊 Score to MAPE Mapping (Public Leaderboard):")
for score in scores:
    mape = ((100 - score) / 25) * reference_cost
    print(f"   Score {score:.2f} → MAPE: {mape:.3f}%")

## 10. Feature Engineering Insights

In [ ]:
# Recreate key engineered features for visualization
vol_cols = [f'Component{i}_fraction' for i in range(1, 6)]
volumes = train[vol_cols].values

# Volume entropy
train['VolumeEntropy'] = -np.sum(volumes * np.log(volumes + 1e-10), axis=1)

# Volume standard deviation
train['VolumeStd'] = train[vol_cols].std(axis=1)

# Weighted Property 1 example
prop1_cols = [f'Component{i}_Property1' for i in range(1, 6)]
props1 = train[prop1_cols].values
train['WeightedProp1'] = np.sum(volumes * props1, axis=1)

print("🔧 Engineered Features Statistics:")
print(f"   Volume Entropy: Mean={train['VolumeEntropy'].mean():.4f}, Std={train['VolumeEntropy'].std():.4f}")
print(f"   Volume Std: Mean={train['VolumeStd'].mean():.4f}, Std={train['VolumeStd'].std():.4f}")
print(f"   Weighted Prop1: Mean={train['WeightedProp1'].mean():.4f}, Std={train['WeightedProp1'].std():.4f}")

In [ ]:
# Visualize engineered features
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Engineered Feature Distributions', fontsize=14, fontweight='bold')

# Volume Entropy
axes[0].hist(train['VolumeEntropy'], bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[0].set_title('Volume Entropy', fontweight='bold')
axes[0].set_xlabel('Entropy')
axes[0].set_ylabel('Frequency')

# Volume Std
axes[1].hist(train['VolumeStd'], bins=50, color='teal', alpha=0.7, edgecolor='black')
axes[1].set_title('Volume Standard Deviation', fontweight='bold')
axes[1].set_xlabel('Std Dev')
axes[1].set_ylabel('Frequency')

# Weighted Property 1
axes[2].hist(train['WeightedProp1'], bins=50, color='coral', alpha=0.7, edgecolor='black')
axes[2].set_title('Weighted Property 1', fontweight='bold')
axes[2].set_xlabel('Value')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 11. Summary & Key Takeaways

In [ ]:
print("="*70)
print("📊 SHELL.AI 2026 - ANALYSIS SUMMARY")
print("="*70)

print("\n🎯 MODEL PERFORMANCE:")
print(f"   Public Leaderboard Score: 83.16 / 100")
print(f"   Estimated MAPE: ~1.83%")
print(f"   Execution Time: ~21 minutes")

print("\n📊 DATASET CHARACTERISTICS:")
print(f"   Training Samples: {len(train):,}")
print(f"   Test Samples: 500")
print(f"   Input Features: 55 (5 fractions + 50 properties)")
print(f"   Target Properties: 10")
print(f"   Engineered Features: ~150+")

print("\n🔧 KEY TECHNIQUES:")
print("   ✓ Physics-inspired feature engineering")
print("   ✓ SHAP-based feature selection (top 50 per target)")
print("   ✓ LightGBM with L1 regression")
print("   ✓ 5-fold cross-validation")
print("   ✓ Physics-based prediction clipping")

print("\n💡 INSIGHTS:")
print("   • Component fractions sum to 1.0 (validated)")
print("   • Blend properties show varying degrees of correlation")
print("   • Predictions remain within training data ranges")
print("   • Non-linear relationships captured through engineered features")

print("\n🌟 IMPACT:")
print("   • Enables rapid blend property estimation")
print("   • Supports sustainable aviation fuel development")
print("   • Contributes to net-zero transition goals")

print("\n" + "="*70)
print("✅ Analysis Complete!")
print("="*70)

## 12. Next Steps & Potential Improvements

### Possible Enhancements:

1. **Advanced Feature Engineering:**
   - Polynomial features for component fractions
   - Interaction terms between different properties
   - Domain-specific mixing rules (Kay's rule, Refutas equation)

2. **Model Improvements:**
   - Hyperparameter tuning (Optuna, Hyperopt)
   - Ensemble methods (stacking multiple models)
   - Neural networks for complex non-linearities
   - Multi-task learning (joint prediction of all properties)

3. **Validation Strategy:**
   - Stratified k-fold by target ranges
   - Out-of-distribution detection
   - Uncertainty quantification

4. **Interpretability:**
   - SHAP waterfall plots for individual predictions
   - Partial dependence plots
   - Feature interaction analysis

5. **Computational Efficiency:**
   - Feature selection optimization
   - Model compression
   - Parallel processing for multi-target training

---

<div style="text-align: center; padding: 20px; background-color: #f0f0f0; border-radius: 10px;">
    <h3>🌍 Shell.ai Hackathon 2026</h3>
    <p><strong>Accelerating the transition to sustainable aviation fuels through AI</strong></p>
    <p>Score: 83.16 | MAPE: ~1.83%</p>
</div>